In [1]:
# conding: utf-8

In [2]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt

In [3]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

### Functions

In [4]:
def sign(a):
    return -1 if a < 0 else 1

def r_(score):
    """ """
    return sign(score) * math.sqrt(abs(score))

def get_stats(index):
    try:
        return (
            STATS_ID["mean"][index],
            STATS_ID["median"][index], 
            STATS_ID["std"][index]
        )
    except:
        return False

### The Data

In [5]:
with pd.HDFStore("../data/train.h5", "r") as data:
    DATA = data.get("train")

In [6]:
DATA_TRAIN, DATA_VALID = train_test_split(DATA, test_size=0.2, random_state=42)

In [7]:
print "ids in data \t\t:", len(DATA.id.unique())
print "ids in data_train \t:" , len(DATA_TRAIN.id.unique())

ids in data 		: 1424
ids in data_train 	: 1424


In [8]:
DATA.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,...,0.775208,NaN,NaN,NaN,-0.414776,NaN,NaN,-2.0,NaN,-0.011753
1,11,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,...,0.025590,NaN,NaN,NaN,-0.273607,NaN,NaN,-2.0,NaN,-0.001240
2,12,0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.155937,1.219439,-0.764516,...,0.151881,NaN,NaN,NaN,-0.175710,NaN,NaN,-2.0,NaN,-0.020940
3,25,0,NaN,NaN,NaN,NaN,NaN,0.178495,NaN,-0.007262,...,1.035936,NaN,NaN,NaN,-0.211506,NaN,NaN,-2.0,NaN,-0.015959
4,26,0,0.176693,-0.025284,-0.057680,0.015100,0.180894,0.139445,-0.125687,-0.018707,...,0.630232,NaN,NaN,NaN,-0.001957,NaN,NaN,0.0,NaN,-0.007338


In [9]:
STATS_ID = DATA.groupby(['id']).agg({'y': [np.mean, np.median, np.std]})['y']

In [10]:
STATS_ID["mean"][6], STATS_ID["median"][6], STATS_ID["std"][6]

(0.00015385225, -0.00010041468, 0.014151469)

In [11]:
MEANS = DATA_TRAIN.mean()

In [12]:
DATA_TRAIN = DATA_TRAIN.fillna(MEANS)

In [13]:
columns = ['technical_30', 'technical_20', 'fundamental_11', 'technical_7', 'technical_21', 'technical_40']

In [14]:
non_used = ['id', 'timestamp', 'y']

In [15]:
tree = RandomForestRegressor(n_estimators=10, criterion="mse", max_features=20)

In [16]:
tree.fit(DATA_TRAIN.drop(non_used, axis=1), DATA_TRAIN.y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=20, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [17]:
DATA_TRAIN.y.isnull().sum()

0

In [18]:
DATA_VALID = DATA_VALID.fillna(MEANS)

In [19]:
tree.score(DATA_VALID.drop(non_used, axis=1), DATA_VALID.y)

-0.19368771160687426

In [20]:
RES = pd.DataFrame({})

In [21]:
RES['id'] = DATA_VALID.id
RES['y' ] = DATA_VALID.y
RES['y_'] = tree.predict(DATA_VALID.drop(non_used, axis=1))

In [22]:
RES = RES.join(STATS_ID, on="id", how='left')

In [23]:
RES.head()

,id,y,y_,mean,median,std
1436942,244,-0.059373,-0.008460,0.000641,0.000127,0.029764
1311584,1331,-0.007967,0.011544,-0.001465,-0.002031,0.028815
659753,1024,-0.003135,0.003108,0.000239,-0.000134,0.017686
1532327,911,-0.012868,0.005787,0.000061,-0.000170,0.011359
560982,253,-0.015441,-0.023081,-0.000280,-0.001434,0.021702


In [24]:
def fun(row):
    return (1 - 2 * row["std"]) * row["y_"] + 2 * row["std"] * row["median"]

In [25]:
RES["prediction"] = RES.apply(fun, axis=1)

In [26]:
r_(r2_score(RES.y, RES.prediction))

-0.41568855559968226

In [27]:
r_(r2_score(RES.y, RES.y_))

-0.4400996609938188